<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/convert-verbs-to-baseform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 pandas
!pip install ipadic
!python -m ipadic download

In [ ]:
import pandas as pd
import MeCab
import ipadic

# MeCabを使って動詞を辞書形に修正する

# 入力CSVファイルと出力CSVファイルのパス
input_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b.csv" # 元のCSVファイル
output_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b-fixed.csv"  # 修正後のCSVファイル

# MeCabの初期化（辞書パス指定なしでunidicを使用）
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)

# CSVファイルを読み込む
try:
    df = pd.read_csv(input_csv)
    print(f"読み込んだCSVの行数: {len(df)}")

     # Extracted_Phraseの動詞を辞書形に修正
    def convert_to_base_form(phrase):
        tokens = tagger.parse(phrase).splitlines()
        base_phrase = []  # 修正後のフレーズ
        for token in tokens:
            if '\t' not in token:  # EOSや空行をスキップ
                continue
            surface, features = token.split('\t')  # 表層形と品詞情報
            features = features.split(',')
            pos = features[0]  # 品詞
            base_form = features[6] if len(features) > 6 and features[6] != '*' else surface
            # 動詞の基本形を使用
            if pos == '動詞':
                base_phrase.append(base_form)
            else:
                base_phrase.append(surface)
        return ''.join(base_phrase)

    # DataFrameのExtracted_Phrase列を変換
    df['Extracted_Phrase'] = df['Extracted_Phrase'].apply(convert_to_base_form)

    # 修正後のデータを新しいCSVファイルに保存
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"修正後のCSVファイルを保存しました: {output_csv}")

except Exception as e:
    print(f"エラーが発生しました: {e}")

In [ ]:

# 行数を確認
input_df = pd.read_csv(input_csv)
output_df = pd.read_csv(output_csv)

print(f"入力CSVの行数: {len(input_df)}")
print(f"出力CSVの行数: {len(output_df)}")

if len(input_df) == len(output_df):
    print("行数が一致しています。処理は正常です。")
else:
    print("行数が一致しません！処理に問題があります。")


In [ ]:
# 比較する行番号（0からカウント）
target_row = 10  # 例：10行目

# 行数チェック
if target_row < len(input_df) and target_row < len(output_df):
    input_row = input_df.iloc[target_row]
    output_row = output_df.iloc[target_row]

    print(f"--- {target_row} 行目の比較 ---")
    print("【入力データ】")
    print(input_row)
    print("\n【出力データ】")
    print(output_row)

    # 差分の確認
    print("\n【差分】")
    for column in input_df.columns:
        if input_row[column] != output_row[column]:
            print(f"カラム: {column} | 入力: {input_row[column]} | 出力: {output_row[column]}")
else:
    print(f"指定した行番号 {target_row} は範囲外です。入力CSV: {len(input_df)} 行, 出力CSV: {len(output_df)} 行")